In [47]:
import pandas as pd
ball_by_ball = pd.read_csv('../resources/train/ball_by_ball_data.csv')
match_data = pd.read_csv('../resources/train/match_data.csv')
bbb_venue = ball_by_ball.merge(match_data, on='match_id')
bbb_venue.head(n=2)
mbm = ball_by_ball.groupby(['match_id', 'inning', 'batting_team', 'bowling_team', 
                            'batsman_id']).agg({'batsman_runs': np.sum}).reset_index()
mbm_venue = mbm.merge(match_data, on='match_id')

In [48]:
import numpy as np
bat_sat = bbb_venue[['batsman_id', 'match_id', 'season', 'bowling_team', 'venue_id', 'batsman_runs']]

bat_sat_agg = bbb_venue.groupby(['batsman_id', 'season', 'bowling_team', 'venue_id']).agg({'batsman_runs': np.sum}).reset_index()
bat_sat_agg.head(n=2)

,batsman_id,season,bowling_team,venue_id,batsman_runs
0,1,2008,DC,4,37
1,1,2008,DC,5,23


In [49]:
bat_run_group = bbb_venue[bbb_venue['extra_runs'] == 0].groupby(['batsman_id', 'season', 'bowling_team', 'venue_id', 'batsman_runs']).agg({'match_id': np.size}).reset_index()
bat_run_group.head(n = 10)

,batsman_id,season,bowling_team,venue_id,batsman_runs,match_id
0,1,2008,DC,4,0,7
1,1,2008,DC,4,1,9
2,1,2008,DC,4,2,3
3,1,2008,DC,4,4,4
4,1,2008,DC,4,6,1
5,1,2008,DC,5,0,18
6,1,2008,DC,5,1,7
7,1,2008,DC,5,2,4
8,1,2008,DC,5,4,2
9,1,2008,DD,2,0,10


In [50]:
for i in [0,1,2,3,4,6]:
    cname = str(i) + "_runs"
    bat_run_group[cname] = bat_run_group.apply(lambda x: x.match_id if x.batsman_runs == i else 0, axis = 1)
    
bat_run_group.head(n=10)

,batsman_id,season,bowling_team,venue_id,batsman_runs,match_id,0_runs,1_runs,2_runs,3_runs,4_runs,6_runs
0,1,2008,DC,4,0,7,7,0,0,0,0,0
1,1,2008,DC,4,1,9,0,9,0,0,0,0
2,1,2008,DC,4,2,3,0,0,3,0,0,0
3,1,2008,DC,4,4,4,0,0,0,0,4,0
4,1,2008,DC,4,6,1,0,0,0,0,0,1
5,1,2008,DC,5,0,18,18,0,0,0,0,0
6,1,2008,DC,5,1,7,0,7,0,0,0,0
7,1,2008,DC,5,2,4,0,0,4,0,0,0
8,1,2008,DC,5,4,2,0,0,0,0,2,0
9,1,2008,DD,2,0,10,10,0,0,0,0,0


In [51]:
bat_run_class = bat_run_group.groupby(['batsman_id', 'season', 'bowling_team', 'venue_id']).agg({'0_runs': np.sum, '1_runs': np.sum, '2_runs': np.sum, '3_runs': np.sum, '4_runs': np.sum, '6_runs': np.sum}).reset_index()
bat_run_class.head(n =10)

,batsman_id,season,bowling_team,venue_id,0_runs,1_runs,2_runs,3_runs,4_runs,6_runs
0,1,2008,DC,4,7,9,3,0,4,1
1,1,2008,DC,5,18,7,4,0,2,0
2,1,2008,DD,2,10,15,3,0,3,0
3,1,2008,DD,5,8,14,2,0,2,1
4,1,2008,KKR,5,6,13,3,0,3,2
5,1,2008,KXI,5,12,17,4,1,2,4
6,1,2008,KXI,9,1,2,0,0,0,0
7,1,2008,MI,1,14,11,1,0,6,1
8,1,2008,MI,5,4,4,2,0,4,1
9,1,2008,RCB,5,5,2,1,0,0,0


In [92]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
#dismissal_kind
dis_deliveries = bbb_venue[bbb_venue['player_dismissed_id'].notnull()][['match_id', 'player_dismissed_id', 'dismissal_kind']]

dis_sparse= mlb.fit_transform(dis_deliveries.apply(lambda x: set([x.dismissal_kind]), axis = 1))
dis_classes = list(mlb.classes_)

dis_del_byplayer = dis_deliveries.merge(pd.DataFrame(dis_sparse, columns = dis_classes, index = dis_deliveries.index), 
                     left_index=True, right_index=True)

mbm_stat = mbm_venue.merge(dis_del_byplayer, left_on=['batsman_id', 'match_id'], right_on=['player_dismissed_id', 'match_id'], 
                how='left')[['batsman_id', 'season', 'bowling_team', 'venue_id', 
                             'match_id', 'player_dismissed_id'] + dis_classes].fillna(0)

mbm_stat['is_dismissed'] = mbm_stat.apply(lambda x: 0 if x.player_dismissed_id==0 else 1, axis = 1)
mbm_stat['innings'] = mbm_stat.apply(lambda x : 1, axis = 1)
mbm_stat.head(n=2)

,batsman_id,season,bowling_team,venue_id,match_id,player_dismissed_id,bowled,caught,caught and bowled,hit wicket,lbw,obstructing the field,retired hurt,run out,stumped,is_dismissed,innings
0,66,2008,RCB,7,2008_1,66.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,73,2008,RCB,7,2008_1,73.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1


In [93]:
agg_dict = {x: np.sum for x in dis_classes + ['is_dismissed', 'innings']}
vbv_stat = mbm_stat.groupby(['batsman_id', 'season', 'bowling_team', 'venue_id']).agg(agg_dict ).reset_index()
vbv_stat.head(n=2)

,batsman_id,season,bowling_team,venue_id,bowled,caught,caught and bowled,hit wicket,lbw,obstructing the field,retired hurt,run out,stumped,is_dismissed,innings
0,1,2008,DC,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,1,2008,DC,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
